In [1]:
import os

In [2]:
%pwd

'c:\\Users\\mkman\\OneDrive\\Desktop\\Trail_and_Error\\End_to_end_Ml_Project\\research'

In [3]:
%cd ..

c:\Users\mkman\OneDrive\Desktop\Trail_and_Error\End_to_end_Ml_Project


In [4]:
%pwd

'c:\\Users\\mkman\\OneDrive\\Desktop\\Trail_and_Error\\End_to_end_Ml_Project'

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: str
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [ ]:
from mlProject.constant import *
from mlProject.utils.common import read_yaml, create_directories


In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN   # This already gives the string value "quality"

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema   # ✅ no `.name`, schema itself is the string
        )
        return model_evaluation_config



In [13]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
from mlProject.utils.common import save_json
import numpy as np
import joblib 

In [14]:


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_squared_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def save_results(self):
        # Load the test dataset
        test_data = pd.read_csv(self.config.test_data_path)

        # Load the trained model
        model = joblib.load(self.config.model_path)

        # Split features (X) and target (y)
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        # Make predictions
        predicted_qualities = model.predict(test_x)

        # Evaluate predictions
        (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

        # Saving metrics as local JSON file
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config= model_evaluation_config)
    model_evaluation_config.save_results()

except Exception as e:
    raise e

[2025-09-01 22:23:51,699: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2025-09-01 22:23:51,724: INFO: common: yaml file: params.yaml loaded succesfully]
[2025-09-01 22:23:51,733: INFO: common: yaml file: schema.yaml loaded succesfully]
[2025-09-01 22:23:51,746: INFO: common: created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'model_evaluation'"